# Item 21: Know How Closures Interact with Variable Scopes

In [1]:
# We can use a helper function as criteria when sorting a list. The helper's return value will be used as the
# value for sorting each item in the list
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [2]:
# The above function works for simple inputs
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


There are three reasons why the previous function operates as expected:
- Python supports *closures*.
- Functions are first-clas objects in Python.
- Python has specific rules for comparing sequences (first compares items at index zero; then, if those are equal, it compares items at index one; if they are still equal, it compares items at index two and so on). This is why the return value from the `helper` closure causes the sort order to have two distinct groups.

In [3]:
# We can use the closure to flip a flag when high-priority items are seen. Then, the function can return the
# flag value after it has been modified by the closure
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found =  True # Seems simple
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [4]:
found = sort_priority2(numbers, group)
print('Found: ', found)
print(numbers)

Found:  False
[2, 3, 5, 7, 1, 4, 6, 8]


Notice that the sorted results are correct, yet the `found` result returned is `False` when it should be `True`. How did this happen?

When we reference a variable in an expression, the Python interpreter traverses the scope to resolve the reference in this order:
1. The current function's scope.
2. Any enclosing scopes (such as other containing functions).
3. The scope of the module that contains the code (also called the *global scope*).
4. The built-in scope (that contains functions like `len` and `str`).

In [5]:
# If none of these places has defined a variable with the referenced name, then a NameError exception is raised
foo = does_not_exist * 5

NameError: name 'does_not_exist' is not defined

Assigning a value to a variable works differently. If the variable is already defined in the current scope, it will just take on the new value. **If the variables doesn't exist in the current scope, Python treats the assignment as a variable definition**. Critically, the scope of the newly defined variable is the function that contains the assignment. 

 This assignment behavior explains the wrong return value of the `sort_priority2` function. The `found` variable is assigned to `True` in the `helper` closure. The closure's assignment is treated as a new variable definition within the helper, not as an assignment within `sort_priority2`.

In [6]:
def sort_priority2(values, group):
    found = False # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True # Scope: 'helper' --> Bad!
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In Python, the `nonlocal` keyword is used that the scope traversal should happen upon assignment for a specific variable name. The only limit is that `nonlocal` won't traverse up to the module-level scope (to avoid polluting globals). 

In [9]:
# Here, we define the same function, now using nonlocal
def sort_priority3(values, group):
    found = False 
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [10]:
found = sort_priority3(numbers, group)
print('Found: ', found)
print(numbers)

Found:  True
[2, 3, 5, 7, 1, 4, 6, 8]


In [11]:
# The author advices to use nonlocal only with simple functions because the side effects of using 
# nonlocal can be hard to follow. When usage of nonlocal starts getting complicated, it's better to wrap our
# state in a helper class
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found =True
            return (0, x)
        return (1, x)    

In [12]:
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True